In [1]:
import numpy as np
from sympy import Matrix, zoo
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)

        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial guess for z (shape (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Objective function coefficients (shape (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint bounds (shape (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64) # Constraint coefficients (shape (m, n))
        self.n = self.A.shape[1]  # Number of variables

        if self.A.shape[0] < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")

    def isFeasible(self, z=None):
        """Check if the solution z is feasible."""
        if z is None:
            z = self.z
        return np.all(np.matmul(self.A, z) <= self.b)

    def __activeAndUnActiveConst(self):
        """Classify active and inactive constraints based on the current z."""
        constraint_values = np.matmul(self.A, self.z)
        tolerance = 1e-8
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]

        self.A1 = self.A[active_indices]
        self.B1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.B2 = self.b[inactive_indices]

    def __isVertex(self):
        """Check if the current solution z is a vertex."""
        self.__activeAndUnActiveConst()
        return self.A1.shape[0] == self.n

    def find_initial_vertex(self):
        """Find an initial vertex for the feasible region."""
        # Check if the initial z is feasible
        if not self.isFeasible():
            raise ValueError("Initial point z is not feasible.")

        # Loop until z corresponds to a vertex
        while not self.__isVertex():
            self.__activeAndUnActiveConst()

            A1 = self.A1
            A2 = self.A2
            b2 = self.B2
            n = self.n

            # Compute the nullspace of A1
            nullspace = np.array(Matrix(A1).nullspace())
            if nullspace.size == 0:
                # Cannot move along nullspace, so z is at a vertex
                break

            # Pick a direction in the nullspace (first vector)
            direction = nullspace[0].astype(np.float64).flatten()
            direction = direction / np.linalg.norm(direction)

            # Compute step size alpha
            temp = np.dot(A2, direction)
            # Replace zeros with a small number to avoid division by zero
            temp = np.where(np.abs(temp) < 1e-8, 1e-8, temp)
            alpha_values = (b2 - np.dot(A2, self.z)) / temp
            # Take only positive alpha
            positive_alphas = [alpha for alpha in alpha_values if alpha > 0]
            if len(positive_alphas) == 0:
                raise ValueError("Cannot find a positive alpha to move towards a vertex.")

            alpha = min(positive_alphas)

            # Update z
            self.z = self.z + alpha * direction

    def find_optimal_vertex(self):
        """Find the optimal vertex for the objective function and return the cost."""
        self.find_initial_vertex()  # Ensure we start at a valid vertex

        while True:
            self.__activeAndUnActiveConst()

            A1 = self.A1
            A2 = self.A2
            b2 = self.B2
            n = self.n

            if A1.shape[0] != n:
                raise ValueError("Active constraint matrix A1 must be square.")

            # Compute beta = (A1.T)^(-1) * c
            try:
                A1_inv = np.linalg.inv(A1)
            except np.linalg.LinAlgError:
                raise ValueError("Active constraint matrix A1 is singular.")

            beta = np.dot(A1_inv.T, self.c)

            # Check if all beta >= 0
            if np.all(beta >= -1e-8):
                # Current vertex is optimal
                final_cost = np.dot(self.c, self.z).item()
                return self.z, final_cost

            # Otherwise, find a direction to move
            # For each row in A1_inv_T
            A1_inv_T = A1_inv.T
            u = None
            for x in A1_inv_T:
                if np.dot(-x, self.c) > 1e-8:
                    u = x
                    break

            if u is None:
                # Cannot find a direction to improve the objective
                final_cost = np.dot(self.c, self.z).item()
                return self.z, final_cost

            # Compute step size alpha
            temp = np.dot(A2, u)
            # Replace zeros with a small number to avoid division by zero
            temp = np.where(np.abs(temp) < 1e-8, 1e-8, temp)
            alpha_values = (np.dot(A2, self.z) - b2) / temp

            # Take only non-negative alpha
            positive_alphas = [alpha for alpha in alpha_values if alpha >= 0]
            if len(positive_alphas) == 0:
                raise ValueError("Problem is unbounded.")

            alpha = min(positive_alphas)

            # Update z
            self.z = self.z - alpha * u

            if not self.isFeasible():
                raise ValueError("Solution became infeasible during iteration.")


In [3]:
cls = SimplexAlgo(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\My Code\1.Dataset.csv')

In [4]:
cls.find_optimal_vertex()

(array([ 0., 10.]), 19.99999999999999)